In [1]:
import numpy as np
import os
import logging

HOME = os.environ['HOME'] + '/'
datapath = f'{HOME}dataset/zhaokedata'
file_list = os.listdir(datapath)
# remove all '._' files
file_list = [f for f in file_list if not f.startswith('._')]

logging.basicConfig(level=logging.INFO)
logging.info(f'file_list: {file_list}')

# INFO:root:file_list: ['200113plc1p2.npy', '200113plc1p2_TD.npy', '200323plc1p1_Names.npy', '200113plc1p2_Names.npy', '200323plc1p1_TD.npy']

# get size of each file
for f in file_list:
    data = np.load(f'{datapath}/{f}', allow_pickle=True)
    logging.info(f'{f}: {data.shape}')
    

INFO:root:file_list: ['200113plc1p2.npy', '200113plc1p2_TD.npy', '200323plc1p1_Names.npy', '200113plc1p2_Names.npy', '200323plc1p1_TD.npy']
INFO:root:200113plc1p2.npy: ()
INFO:root:200113plc1p2_TD.npy: (1288, 1288)
INFO:root:200323plc1p1_Names.npy: (1208,)
INFO:root:200113plc1p2_Names.npy: (1288,)
INFO:root:200323plc1p1_TD.npy: (1208, 1208)


In [6]:
data = np.load(f'/home/zihan/codes/zhaokedata/200113plc1p21.npy', allow_pickle=True)
data

array({'AB': [[125.0, 141.0, 91.0]], 'P1': [[116.0, 260.0, 123.0], [119.0, 259.0, 121.0], [109.0, 261.0, 118.0], [112.0, 268.0, 119.0]], 'ABa': [[115.0, 143.0, 98.0], [115.0, 143.0, 96.0], [107.0, 139.0, 97.0], [105.0, 137.0, 101.0], [106.0, 120.0, 95.0], [108.0, 109.0, 86.0], [110.0, 113.0, 91.0], [117.0, 114.0, 96.0], [127.0, 111.0, 95.0], [136.0, 119.0, 106.0]], 'ABp': [[131.0, 145.0, 107.0], [145.0, 157.0, 114.0], [143.0, 157.0, 120.0], [139.0, 172.0, 130.0], [141.0, 177.0, 140.0], [157.0, 187.0, 148.0], [160.0, 194.0, 133.0], [167.0, 195.0, 131.0], [170.0, 201.0, 132.0], [163.0, 201.0, 130.0]], 'EMS': [[96.0, 233.0, 109.0], [93.0, 230.0, 103.0], [88.0, 225.0, 101.0], [85.0, 217.0, 102.0], [82.0, 216.0, 99.0], [70.0, 207.0, 94.0], [64.0, 204.0, 90.0], [60.0, 200.0, 102.0], [71.0, 214.0, 128.0], [82.0, 207.0, 124.0]], 'P2': [[117.0, 270.0, 121.0], [124.0, 278.0, 121.0], [122.0, 293.0, 132.0], [122.0, 298.0, 134.0], [125.0, 299.0, 131.0], [125.0, 292.0, 131.0], [130.0, 286.0, 133.0],

In [27]:
import numpy as np
import random
from scipy.linalg import fractional_matrix_power

B = np.array(
    [
        [0, 0, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [0, 0, 2, 2, 0],
        [0, 1, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 0, 3, 3, 0],
    ]
)

# A = 0 B
#     B^T 0

# 获取 A 的转置
B_T = B.T

m, n = B.shape
upper_zeros = np.zeros((m, m))
right_zeros = np.zeros((n, n))

A = np.vstack((np.hstack((upper_zeros, B)), np.hstack((B_T, right_zeros)))) # (n+m) * (n+m), N := n+m
# print(A)

# A= [[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
#     [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
#     [0. 0. 0. 0. 0. 0. 0. 0. 3. 3. 0.]
#     [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
#     [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
#     [1. 1. 2. 1. 0. 3. 0. 0. 0. 0. 0.]
#     [1. 0. 2. 0. 0. 3. 0. 0. 0. 0. 0.]
#     [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]

D = np.diag(np.sum(A, axis=1))

L = D - A

# L
k = 3 ## number of clusters
cut = [] ## 1 * k, cut[i] = all weights across cluster i
asso = [] ## 1 * k, asso[i] = all weights that one side is in cluster i

Y = [] ## N * k, Y[i][j] = 1 if node i is in j

# random a Y
N = n + m
Y = np.zeros((N, k))

# seed 2024

for i in range(N):
    j = random.randint(0, k-1)
    Y[i][j] = 1

tmp = np.dot(Y.T, np.dot(D, Y))

Z = np.dot(Y, fractional_matrix_power(tmp, -0.5))

lfs = np.trace(np.dot(Z.T, np.dot(L, Z)))

# (Y^T D Y)^(-1)Y^T L Y

# rfs = np.trace(np.dot(np.linalg.inv(tmp), np.dot(Y.T, np.dot(L, Y))))

def target(Y, D, L):
    tmp = np.dot(Y.T, np.dot(D, Y))
    return np.trace(np.dot(np.linalg.inv(tmp), np.dot(Y.T, np.dot(L, Y))))

def minimize(Y, A):
    D = np.diag(np.sum(A, axis=1))
    L = D - A
    return target(Y, D, L)

rfs = minimize(Y, A)

ld = np.trace(np.dot(L, np.linalg.inv(D)))

lfs, rfs, ld, tmp


(2.391304347826087,
 2.391304347826087,
 11.0,
 array([[ 2.,  0.,  0.],
        [ 0., 23.,  0.],
        [ 0.,  0., 11.]]))

In [35]:
def target(Y, D, L):
    tmp = np.dot(Y.T, np.dot(D, Y))
    try:
        tmp_inv = np.linalg.inv(tmp)
    except np.linalg.LinAlgError:
        return 1e9  # 当矩阵不可逆时返回 1e9
    return np.trace(np.dot(tmp_inv, np.dot(Y.T, np.dot(L, Y))))

def minimize(Y, A):
    D = np.diag(np.sum(A, axis=1))
    L = D - A
    return target(Y, D, L)

# 穷举所有的 Y
min_value = 1e9

for i in range(k ** N):
    Y = np.zeros((N, k))
    for j in range(N):
        Y[j][i // (k ** j) % k] = 1
    
    value = minimize(Y, A)
    if value < min_value:
        min_value = value
        min_Y = Y
        
min_value, min_Y

(0.41025641025641024,
 array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]]))

In [36]:
min_Z = np.dot(min_Y, fractional_matrix_power(np.dot(min_Y.T, np.dot(D, min_Y)), -0.5))
F = np.dot(fractional_matrix_power(D, 0.5), min_Z)

F

array([[0.        , 0.2773501 , 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.        , 0.39223227, 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.70710678, 0.        , 0.        ],
       [0.        , 0.48038446, 0.        ],
       [0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.        , 0.5547002 , 0.        ],
       [0.        , 0.48038446, 0.        ],
       [0.5       , 0.        , 0.        ]])